In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 기본 URL 설정
base_url = "https://statiz.sporki.com/schedule/"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# 연도와 월 설정
year = 2025
months = range(3, 8)  # 3월부터 11월까지

# 데이터를 저장할 리스트
all_data = []

for month in months:
    print(f"Fetching data for {year}-{month}...")
    params = {"year": year, "month": month}
    response = requests.get(base_url, headers=headers, params=params)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        
        # 주별 데이터 탐색
        rows = soup.select("table tbody tr")
        for row in rows:
            # 각 요일별 데이터 처리 (td:nth-child(2) ~ td:nth-child(8))
            for day_idx in range(2, 9):  # 2부터 8까지는 월~일을 나타냄
                day_cell = row.select_one(f"td:nth-child({day_idx})")
                
                if day_cell:
                    # 날짜 추출
                    date_tag = day_cell.select_one("span.day")
                    if date_tag:
                        day = date_tag.text.strip()
                        date = f"{year}-{month:02d}-{int(day):02d}"
                        
                        # 해당 날짜의 경기 리스트 추출
                        games = day_cell.select("div.games > ul > li > a")
                        for game in games:
                            teams = game.find_all("span", class_="team")
                            

                            if len(teams) == 2:
                                team1 = teams[0].text.strip()
                                team2 = teams[1].text.strip()


                                # 데이터 저장
                                all_data.append({
                                    "Date": date,
                                    "Team1": team1,
                                    "Team2": team2,

                                })
    else:
        print(f"Failed to fetch data for {year}-{month}: HTTP {response.status_code}")

# 모든 데이터를 데이터프레임으로 변환
df = pd.DataFrame(all_data)

# 결과 출력 및 저장
print(df)
df.to_csv("statiz_game_schedule_2025.csv", index=False, encoding="utf-8-sig")
print("2024년 전체 데이터가 statiz_game_schedule_2025.csv로 저장되었습니다.")


Fetching data for 2025-3...
Fetching data for 2025-4...
Fetching data for 2025-5...
Fetching data for 2025-6...
Fetching data for 2025-7...
           Date Team1 Team2
0    2025-03-22    롯데    LG
1    2025-03-22    두산   SSG
2    2025-03-22    키움    삼성
3    2025-03-22    한화    KT
4    2025-03-22    NC   KIA
..          ...   ...   ...
530  2025-07-31    KT    LG
531  2025-07-31    키움   SSG
532  2025-07-31    NC    롯데
533  2025-07-31    두산   KIA
534  2025-07-31    삼성    한화

[535 rows x 3 columns]
2024년 전체 데이터가 statiz_game_schedule_2025.csv로 저장되었습니다.
